In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ydata_profiling as pp
import seaborn as sns
import warnings
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.ensemble import AdaBoostRegressor, VotingRegressor, GradientBoostingRegressor, StackingRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectPercentile, mutual_info_regression, VarianceThreshold, SelectFromModel
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, LinearRegression,LogisticRegression
from sklearn.metrics import make_scorer, mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import RobustScaler

data = pd.read_csv('../triumphventure/data/clean_data.csv', encoding= 'unicode_escape')

In [3]:
# Drop the unnamed column
data = data.drop(columns=['Unnamed: 0'])

In [5]:
#X_strong_features_test = X_test_new.drop(columns=list(weak_features))
y_new = data["status"].astype(int)
X_new = data.drop(columns=["status"])
from sklearn.model_selection import train_test_split
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)



TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'columns'

In [21]:
class columnDropperTransformer(TransformerMixin, BaseEstimator):
    def __init__(self):
        self.weak_features = ['Industry_Group_Gaming', 'Industry_Group_Professional Services',
       'Industry_Group_Financial Services', 'Industry_Group_Advertising',
       'Industry_Group_Mobile', 'Industry_Group_Media and Entertainment',
       'Industry_Group_Privacy and Security', 'Industry_Group_Education',
       'Industry_Group_Content and Publishing',
       'Industry_Group_Sales and Marketing',
       'Industry_Group_Manufacturing',
       'Industry_Group_Consumer Electronics',
       'Industry_Group_Data and Analytics',
       'Industry_Group_Transportation', 'Industry_Group_Hardware',
       'Industry_Group_Clothing and Apparel',
       'Industry_Group_Real Estate', 'Industry_Group_Design',
       'Industry_Group_Food and Beverage',
       'Industry_Group_Travel and Tourism', 'Industry_Group_Platforms',
       'Industry_Group_Sports', 'Industry_Group_Events',
       'Industry_Group_Consumer Goods',
       'Industry_Group_Administrative Services', 'Industry_Group_Energy',
       'Industry_Group_Sustainability',
       'Industry_Group_Messaging and Telecommunication',
       'Industry_Group_Navigation and Mapping',
       'Industry_Group_Agriculture and Farming',
       'Industry_Group_Natural Resources',
       'Industry_Group_Government and Military', 'Industry_Group_Apps',
       'Industry_Group_Artificial Intelligence',
       'Industry_Group_Science and Engineering']
    
    def fit(self,X,y=None):
        self.encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
        self.encoder.fit(X)
        self.feature_names_ = self.encoder.get_feature_names_out(['country_code','Industry_Group'])
        return self
    def transform(self,X,y=None):
        encoded = self.encoder.transform(X)
        # Create a DataFrame with the transformed data and feature names
        transformed_df = pd.DataFrame(data=encoded, columns=self.feature_names_)
        return transformed_df.drop(self.weak_features, axis=1)

In [22]:
preproc_numerical= make_pipeline(
    SimpleImputer(),
    RobustScaler()
)

preproc_categorical_Industry_Group_country = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    columnDropperTransformer()
)

preproc = make_column_transformer(
    (preproc_numerical, make_column_selector(dtype_include=["int64", "float64"])),
    (preproc_categorical_Industry_Group_country, make_column_selector(dtype_include=["object"]))
)


In [23]:
pipe_baseline = make_pipeline(preproc, RandomForestClassifier(random_state=42))

In [24]:
pd.DataFrame(preproc.fit_transform(X_train))

/Users/deepatamraparani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,13.843750,7.0,3.623239,0.156724,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.191172,0.0,0.000000,-0.738625,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.135923,0.0,0.000000,-0.698686,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.493021,1.0,3.772887,1.092518,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.125000,0.0,0.000000,-0.125379,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29424,-0.182292,0.0,0.000000,0.544489,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29425,1.130208,2.0,2.036972,0.524267,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29426,-0.204167,1.0,0.107394,-0.384732,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29427,-0.104167,0.0,0.000000,0.353893,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
preproc

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x17accd510>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('columndroppertransformer',
                                                  columnDropperTransformer())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x17b82b760>)])

In [31]:
pipe_baseline.fit(X_train,y_train)

/Users/deepatamraparani/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x17accd510>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('columndroppertransformer',
                                                                   columnDropperTransformer())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x17b82b760>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [33]:
y_pred = pipe_baseline.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 0, 0])